In [ ]:
import geopandas as gpd
import requests, zipfile, io, os

# Crear carpeta destino si no existe
os.makedirs("data/municipios_conabio", exist_ok=True)

# URL del shapefile municipal de CONABIO
url = "http://www.conabio.gob.mx/informacion/gis/maps/geo/mun23gw.zip"

# Descargar el archivo
r = requests.get(url)
if r.status_code == 200:
    print("✅ Descarga completada.")
else:
    print("⚠️ Error al descargar el archivo.")

# Extraer los archivos ZIP
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("data/municipios_conabio")

# Verificar que el archivo .shp exista
for file in os.listdir("data/municipios_conabio"):
    if file.endswith(".shp"):
        shp_path = os.path.join("data/municipios_conabio", file)
        print(f"✅ Shapefile encontrado: {shp_path}")

# Leer el shapefile automáticamente detectado
municipios = gpd.read_file(shp_path)
print("✅ Shapefile cargado correctamente")
municipios.head()


In [ ]:
# 🧩 Importar librerías necesarias
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import zipfile, os

# Cargar shapefile de municipios
shp_file_path = "/content/data/municipios_conabio/mun23gw.shp"
municipios = gpd.read_file(shp_file_path)
print("✅ Shapefile cargado correctamente.")
print("CRS:", municipios.crs)

# Reproyectar a EPSG:4326 para compatibilidad
municipios = municipios.to_crs("EPSG:4326")
print("📍 CRS convertido a EPSG:4326")

In [ ]:
import numpy as np

# Crear DataFrame ficticio
np.random.seed(42)
num_observations = 3214 # Set the number of observations

# Create a list of 300 unique observer names
num_unique_observers = 300
base_observer_names = [f"Entrenador_{i}" for i in range(1, num_unique_observers + 1)]

# Introduce some typographical errors in a subset of names
typo_names = []
import random
import string

def introduce_typo(name):
    if len(name) < 3:
        return name
    typo_type = random.choice(["swap", "delete", "insert"])
    name_list = list(name)

    if typo_type == "swap":
        if len(name_list) > 1:
            i, j = random.sample(range(len(name_list)), 2)
            name_list[i], name_list[j] = name_list[j], name_list[i]
    elif typo_type == "delete":
        i = random.choice(range(len(name_list)))
        del name_list[i]
    elif typo_type == "insert":
        i = random.choice(range(len(name_list) + 1))
        name_list.insert(i, random.choice(string.ascii_lowercase))
    return "".join(name_list)

# Apply typos to a percentage of unique names
num_typos = int(num_unique_observers * 0.2) # Introduce typos in 20% of names
typo_indices = np.random.choice(num_unique_observers, num_typos, replace=False)

observer_names_with_typos = base_observer_names.copy()
for i in typo_indices:
    observer_names_with_typos[i] = introduce_typo(observer_names_with_typos[i])


# Randomly sample from the list of observer names (with and without typos) for the observations
observer_list = np.random.choice(observer_names_with_typos, num_observations, replace=True)


zubat_data = pd.DataFrame({
    "scientificName": ["Zubat"]*num_observations,
    "individualCount": np.random.randint(1, 8, num_observations),
    "decimalLatitude": np.random.uniform(18.0, 29.0, num_observations),
    "decimalLongitude": np.random.uniform(-115.0, -97.0, num_observations),
    "observer": observer_list, # Use the generated observer list
    "observationDate": pd.date_range("1932-01-01", periods=num_observations).strftime("%Y-%m-%d")
})

# Add new scientific columns with 60-90% NaN values
num_rows = len(zubat_data)
num_nan = int(num_rows * np.random.uniform(0.6, 0.9)) # Random number of NaNs between 60% and 90%

for col in ["altitud", "profundidad", "precision", "scientific_metric_1", "scientific_metric_2"]:
    # Generate random data, and replace a percentage with NaNs
    data = np.random.rand(num_rows) * 100 # Example random data
    nan_indices = np.random.choice(num_rows, num_nan, replace=False)
    data[nan_indices] = np.nan
    zubat_data[col] = data

zubat_data.head()

In [ ]:
zubat_data.to_csv("zubat_data.csv", index=False)

In [ ]:
df = pd.read_csv("zubat_data.csv")
df.head()

In [ ]:
df.shape

In [ ]:
geometry = [Point(xy) for xy in zip(zubat_data["decimalLongitude"], zubat_data["decimalLatitude"])]
gdf_zubat = gpd.GeoDataFrame(zubat_data, geometry=geometry, crs="EPSG:4326")

# Vista rápida
gdf_zubat.head()


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
municipios.plot(ax=ax, color='whitesmoke', edgecolor='gray')
gdf_zubat.plot(ax=ax, color='purple', markersize=40, alpha=0.6)
plt.title("🦇 Distribución de avistamientos de Zubat en México")
plt.show()


In [ ]:
# Realizar unión espacial
gdf_zubat_mun = gpd.sjoin(gdf_zubat, municipios, how='left', predicate='within')

# Mostrar los campos relevantes
gdf_zubat_mun[['scientificName', 'observer', 'NOMGEO', 'NOM_ENT']].head()


In [ ]:
# Agrupar por municipio
zubat_count = gdf_zubat_mun.groupby('NOMGEO')['scientificName'].count().reset_index()
zubat_count.rename(columns={'scientificName': 'n_observaciones'}, inplace=True)

# Unir con shapefile
mun_joined = municipios.merge(zubat_count, on='NOMGEO', how='left').fillna(0)

# Graficar mapa de abundancia
ax = mun_joined.plot(
    column='n_observaciones',
    cmap='Purples',
    legend=True,
    figsize=(12,10),
    edgecolor='black'
)
plt.title("🦇 Abundancia de Zubat por municipio")
plt.show()


-------------------------------
-------------------------------

# 🧭 Bloque 10A – Visualización SIG Básica: Exploración Espacial y Primeras Visualizaciones 🗺️

## 🎯 Objetivo
Aprender a cargar, explorar y visualizar shapefiles en Python.  
Integraremos un dataset ficticio de observaciones de **Zubat 🦇** para entender cómo conectar bases CSV con datos geográficos.

## 🧰 Librerías
- geopandas
- pandas
- matplotlib
- seaborn
- zipfile / requests
- shapely


In [ ]:
# 🧩 Importar librerías necesarias
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile, requests, io, os
from shapely.geometry import Point


## 🌐 1. Cargar el shapefile municipal desde archivo o desde URL

Podemos:
1. **Usar un shapefile local**, por ejemplo `/content/mun23gw.zip`.
2. **Descargar automáticamente** desde CONABIO:
   - URL: http://www.conabio.gob.mx/informacion/gis/maps/ccl/mun23gw_c.zip


In [ ]:
# Definir ruta y carpeta
local_zip = "/content/mun23gw.zip"
extract_path = "/content/data/municipios_conabio"
os.makedirs(extract_path, exist_ok=True)

# Si no existe el ZIP local, descargar desde URL
if not os.path.exists(local_zip):
    print("📥 Descargando shapefile desde CONABIO...")
    url = "http://www.conabio.gob.mx/informacion/gis/maps/ccl/mun23gw_c.zip"
    r = requests.get(url)
    open(local_zip, "wb").write(r.content)

# Descomprimir el shapefile
with zipfile.ZipFile(local_zip, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Detectar archivo .shp
for file in os.listdir(extract_path):
    if file.endswith(".shp"):
        shp_path = os.path.join(extract_path, file)
        print(f"✅ Shapefile detectado: {shp_path}")

# Leer shapefile
municipios = gpd.read_file(shp_path)
municipios = municipios.to_crs("EPSG:4326")
print("✅ Shapefile cargado correctamente.")


## 🗂️ 2. Explorar los atributos del shapefile (.dbf)


In [ ]:
print("📋 Columnas disponibles en el shapefile:")
for col in municipios.columns:
    print(f"- {col}")

print("\n🧭 Sistema de referencia de coordenadas (CRS):", municipios.crs)
municipios.head()


## 3. Carga de datos de CSV

In [ ]:
zubat_df = pd.read_csv("zubat_data.csv")
zubat_df.head()

## 🧮 4. Análisis exploratorio (EDA) del CSV


In [ ]:
zubat_df.info()
print("\n🔍 Estadísticas básicas:")
print(zubat_df.describe())

print("\n🎯 Observaciones por entrenador:")
print(zubat_df["observer"].value_counts())

## 🌎 5. Convertir CSV a GeoDataFrame


In [ ]:
geometry = [Point(xy) for xy in zip(zubat_df["decimalLongitude"], zubat_df["decimalLatitude"])]
gdf_zubat = gpd.GeoDataFrame(zubat_df, geometry=geometry, crs="EPSG:4326")
gdf_zubat.head()


## 🗺️ 6. Visualización 1 – Mapa base con los registros de Zubat


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
municipios.plot(ax=ax, color='whitesmoke', edgecolor='gray')
gdf_zubat.plot(ax=ax, color='purple', markersize=40, alpha=0.6)
plt.title("🦇 Distribución de Zubat en México")
plt.show()


## 🎨 7. Visualización 2 – Diferenciar por entrenador (usando `hue`)


In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
municipios.plot(ax=ax, color='lightgray', edgecolor='white')
sns.scatterplot(
    data=gdf_zubat,
    x='decimalLongitude', y='decimalLatitude',
    hue='observer', palette='Set2', s=100
)
plt.title("🦇 Avistamientos de Zubat por entrenador")
plt.show()


## 💜 8. Visualización 3 – Conteo de observaciones por entrenador


In [ ]:
sns.countplot(data=zubat_df, x='observer', palette='Purples')
plt.title("Número de observaciones de Zubat por entrenador")
plt.show()


## 🕒 9. Visualización 4 – Evolución temporal de observaciones


In [ ]:
zubat_df.groupby("observationDate")["individualCount"].sum().plot(figsize=(8,5))
plt.title("Evolución temporal de observaciones de Zubat")
plt.ylabel("Individuos registrados")
plt.xlabel("Fecha")
plt.show()


## 🗺️ 10. Visualización 5 – Mapa de calor (densidad)


In [ ]:
from folium import Map
from folium.plugins import HeatMap

coords = gdf_zubat[["decimalLatitude", "decimalLongitude"]].values.tolist()
m = Map(location=[23, -102], zoom_start=5)
HeatMap(coords, radius=10).add_to(m)
m


## 🧩 Ejercicios para el estudiante

1. Agrega otro Pokémon (Golbat) y compáralo con Zubat.
2. Cambia el color de los entrenadores según su frecuencia.
3. Crea un gráfico de barras agrupado por entrenador y mes.
4. Guarda el mapa en un archivo HTML.
5. Escribe una breve interpretación de los patrones espaciales.


# 🔥 Bloque 10B – Visualización SIG Avanzada: Operaciones y Análisis Ecológico 🌍

## 🎯 Objetivo
Aplicar técnicas avanzadas de análisis espacial (SIG) usando GeoPandas y visualizar métricas ecológicas como abundancia, riqueza y distribución.

## 🧰 Librerías
- geopandas
- pandas
- matplotlib
- seaborn
- shapely
- folium


## 🧭 1. Unión espacial entre puntos (Zubat) y municipios

Permite asociar cada observación con el municipio y estado al que pertenece.


In [ ]:
# Unión espacial
gdf_zubat_mun = gpd.sjoin(gdf_zubat, municipios, how="left", predicate="within")

# Mostrar resultado parcial
gdf_zubat_mun[["scientificName", "observer", "NOMGEO", "NOM_ENT"]].head()


## 🧮 2. Cálculo de abundancia total por municipio

La abundancia representa el número total de individuos observados en un área.


In [ ]:
abund_mun = gdf_zubat_mun.groupby("NOMGEO")["individualCount"].sum().reset_index()
abund_mun.rename(columns={"individualCount": "abundancia"}, inplace=True)

mun_abund = municipios.merge(abund_mun, on="NOMGEO", how="left").fillna(0)

ax = mun_abund.plot(column="abundancia", cmap="Purples", legend=True, figsize=(10,8), edgecolor="gray")
plt.title("🦇 Abundancia total de Zubat por municipio")
plt.show()


## 📊 3. Abundancia promedio por estado

Nos permite identificar qué estados registran más individuos en promedio.


In [ ]:
abund_estado = gdf_zubat_mun.groupby("NOM_ENT")["individualCount"].mean().reset_index()
plt.figure(figsize=(10,5))
sns.barplot(data=abund_estado, x="NOM_ENT", y="individualCount", palette="plasma")
plt.xticks(rotation=90)
plt.title("Promedio de abundancia por estado")
plt.ylabel("Promedio de individuos")
plt.show()


## 🗺️ 4. Selección y visualización de un estado específico

Ejemplo: filtrar solo **Sonora** para visualizar sus registros.


In [ ]:
estado_sel = municipios[municipios["NOM_ENT"] == "Sonora"]
ax = estado_sel.plot(color="whitesmoke", edgecolor="black", figsize=(8,8))
gdf_zubat.plot(ax=ax, color="red", markersize=60)
plt.title("Avistamientos de Zubat en el estado de Sonora")
plt.show()

## 🧩 5. Filtrado de municipios con alta abundancia (>10 individuos)


In [ ]:
top_mun = abund_mun[abund_mun["abundancia"] > 10]
municipios_top = municipios[municipios["NOMGEO"].isin(top_mun["NOMGEO"])]

ax = municipios.plot(color="whitesmoke", edgecolor="gray", figsize=(10,8))
municipios_top.plot(ax=ax, color="purple")
plt.title("Municipios con más de 10 observaciones de Zubat")
plt.show()



## 🌳 6. Riqueza de especies (si se agregan otros Pokémon)
La riqueza indica cuántas especies distintas hay por municipio.


In [ ]:
riqueza = gdf_zubat_mun.groupby("NOMGEO")["scientificName"].nunique().reset_index()
riqueza.rename(columns={"scientificName":"riqueza_especies"}, inplace=True)
mun_riqueza = municipios.merge(riqueza, on="NOMGEO", how="left").fillna(0)

ax = mun_riqueza.plot(column="riqueza_especies", cmap="viridis", legend=True, figsize=(10,8))
plt.title("🌿 Riqueza de especies Pokémon por municipio")
plt.show()


## 🔥 7. Mapa de calor ponderado por abundancia
Los mapas de calor muestran las áreas con mayor densidad de observaciones.


In [ ]:
from folium import Map
from folium.plugins import HeatMap

m = Map(location=[23, -102], zoom_start=5, tiles="CartoDB positron")
HeatMap(
    gdf_zubat_mun[["decimalLatitude", "decimalLongitude", "individualCount"]].values.tolist(),
    radius=15, blur=20, min_opacity=0.4
).add_to(m)
m

## 📍 8. Cálculo de centroides de distribución

Permite representar el punto medio de distribución de registros por estado.


In [ ]:
from shapely.ops import unary_union

centroides = gdf_zubat_mun.groupby("NOM_ENT")["geometry"].apply(lambda x: unary_union(x).centroid)
centroides_gdf = gpd.GeoDataFrame(centroides, geometry="geometry", crs="EPSG:4326")

ax = municipios.plot(color="whitesmoke", edgecolor="gray", figsize=(8,8))
centroides_gdf.plot(ax=ax, color="blue", markersize=60)
plt.title("Centroides de distribución por estado")
plt.show()

## 🧮 9. Zonas de influencia (Buffers de hábitat potencial)
Creamos áreas circulares alrededor de cada observación de Zubat para simular su rango de vuelo (~0.5° ≈ 55 km).


In [ ]:
buffer = gdf_zubat.copy()
buffer["geometry"] = buffer.buffer(0.5)

ax = municipios.boundary.plot(color="gray", figsize=(10,8))
buffer.plot(ax=ax, color="lightblue", alpha=0.4, edgecolor="blue")
plt.title("Zonas potenciales de hábitat (buffers de 55 km)")
plt.show()


## 🌿 10. Mapa combinado: abundancia + buffers
Combina la información de abundancia municipal con zonas de influencia.


In [ ]:
ax = mun_abund.plot(column="abundancia", cmap="YlGnBu", legend=True, figsize=(10,8))
buffer.boundary.plot(ax=ax, color="purple", alpha=0.5)
plt.title("🦇 Abundancia de Zubat con zonas potenciales de hábitat")
plt.show()


## 🧪 Ejercicios prácticos para el estudiante

1. Selecciona otro estado (ej. Oaxaca) y repite el análisis de abundancia.  
2. Genera un mapa combinado de abundancia + riqueza.  
3. Calcula la abundancia promedio por entrenador.  
4. Crea un buffer más grande (1°) y compara la extensión del hábitat.  
5. Exporta el GeoDataFrame de abundancia a `zubat_abundancia.geojson`.  
6. Calcula los centroides de los municipios con más registros.  
7. Analiza si existe correlación entre `individualCount` y la latitud.  
8. Crea un mapa que use `alpha` para representar esfuerzo de muestreo.  
9. Usa un colormap distinto (ej. `'magma'`, `'coolwarm'`) y compara resultados.  
10. Escribe una breve conclusión ecológica de la distribución espacial de Zubat.


## 💬 Reflexión final

> ¿Qué patrones espaciales observas en la abundancia y riqueza?  
> ¿Qué estados concentran más registros?  
> ¿Cómo cambia el patrón si alteras la escala de análisis (municipal vs estatal)?  
>
> 🌿 *“Un mapa ecológico no solo representa el espacio, sino la historia de las interacciones que lo habitan.”*  
